kali ini akan dibangun sebuah chain management untuk sistem LLM. Terdapat beberapa input yang akan menjadi pipeline untuk LLM. 

Saya akan mencoba mempersiapkan input dari data yang telah dimasukkan atau di-input pada chroamDB. 

Query dari user untuk LLM ini juga akan dijadikan sebagai query untuk mengambil data yang relevan dari ChroamDB

### Beberapa hal yang perlu di define
* Model LLM 
* Model Embedding untuk mengubah teks menjadi vector
* Memasukkan vector kedalam ChromaDB dengan langchain_chroma
* memasukkan file external data dari file xlsx ke dalam chromadb dengan document_loader

#### Model LLM

In [1]:
from langchain_community.chat_models import ChatOllama

/Users/a/Programming/Langchain-Project/my-env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
llm_model = ChatOllama(
    model='llama3:8b',
    temperature=0
)

#### Model Embeddings

In [5]:
# memanggil Indobert dari transformer
from transformers import BertTokenizer, AutoModel

tokenizer = BertTokenizer.from_pretrained("Indobenchmark/indobert-base-p1")
model = AutoModel.from_pretrained("indobenchmark/indobert-base-p1")

In [6]:
# membuat class model embdding
from typing import List 
from langchain_core.embeddings import Embeddings
import torch

class IndoBertEmbeddings(Embeddings):
    def __init__(self, model_name="indobenchmark/indobert-base-p1"):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.model.eval()


    def _generate_embedding(self, text: str) -> List[float]:
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

        with torch.no_grad():
            outputs = self.model(**inputs)

        # polling token menjadi satu vector kalimat
        token_embeddings = outputs.last_hidden_state

        # melakukan mean polling
        sentence_embeddings = token_embeddings.mean(dim=1)

        # konversi ke list python
        return sentence_embeddings.squeeze().tolist()
    

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return [self._generate_embedding(text) for text in texts]
    

    # metode untuk pencarian query pada chroma 
    def embed_query(self, text: str) -> List[float]:
        return self._generate_embedding(text)


### Feeding data to chromaDB dengan menggunakan model class Indobert Embeddings

In [7]:
from langchain_chroma import Chroma

In [8]:
embeddings = IndoBertEmbeddings()

In [9]:
vectore_store = Chroma(
    collection_name="chroma-with-langchain",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db"
)

### Dataset Section 

In [10]:
from langchain_core.documents import Document

data_teks = [Document(
    page_content="Nama pembuat sistem ini adalah Romi Wahyudi Hasibuan",
    metadata={"sumber":'wikipedia', "tahun":2025}
)]

In [11]:
vectore_store.add_documents(data_teks)

['9f3e8296-002b-47ef-a0eb-cf2ea436e2e7']

Test Query atau proses Retrieve pada ChromaDB

In [12]:
query = "siapa nama yang membuat sistem ini"

In [13]:
result = vectore_store.similarity_search(query=query, k=2)

In [14]:
for doc in result:
    print(doc.page_content)

Nama pembuat sistem ini adalah Romi Wahyudi Hasibuan
Nama pembuat sistem ini adalah Romi Wahyudi Hasibuan


Note. Proses Retreive belum sepenuhnya maksimal, karena pada proses diatas, proses retrieval belum maksimal memberikan data yang relevan. Hal ini nanti bisa di uji dengan retrieval generation testing

### Chains Template atau prompt dengan model LLM

In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [16]:
prompt = ChatPromptTemplate.from_template(
    """Kamu adalah asisten chatbot yang membantu. 
    Tugasmu adalah menjawab pertanyaan berikut secara langsung dalam Bahasa Indonesia. 
    Meskipun pertanyaan diberikan dalam Bahasa Inggris atau bahasa lain, kamu WAJIB menjawabnya dalam Bahasa Indonesia.
    
    Pertanyaan: {input}
    Jawaban:"""
)

In [15]:
interface_user = "what is novelty"

In [16]:
chain = prompt | llm_model | StrOutputParser()

In [17]:
respone = chain.invoke({"input": interface_user})
print(respone)

Inovasi! Novelty dalam konteks ini berarti sesuatu yang baru, unik, dan tidak biasa. Dalam arti luas, novelty dapat merujuk pada gagasan, ide, atau produk yang belum pernah ada sebelumnya dan memiliki potensi untuk menjadi inovatif dan berbeda dari apa yang telah diketahui sebelumnya.


### Chain semua proses (Setup dan retrieval -> prompt -> llm model -> parser output)

jadikan vector store sebagai retreiever

In [17]:
retriever = vectore_store.as_retriever()

karena sudah menggunakan konteks, dimana query atau pertanyaan dari user akan dijadikan sebagai query input untuk proses retriever. Maka dibutuhkan handling input more than one atau secara parallel

In [18]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

In [19]:
prompt = ChatPromptTemplate.from_template(
    """Kamu adalah asisten cerdas. Jawablah pertanyaan pengguna berdasarkan konteks berikut ini saja.
    
    KONTEKS:
    {context}
    
    INSTRUKSI TAMBAHAN:
    1. Jawablah pertanyaan berdasarkan KONTEKS di atas.
    2. Kamu WAJIB menjawab dalam Bahasa Indonesia, terlepas dari bahasa apa pertanyaan itu diberikan.
    3. Jika jawaban tidak ada di dalam konteks, katakan "Maaf, informasi tidak ditemukan dalam dokumen."
    
    PERTANYAAN: {question}
    JAWABAN:"""
)

In [20]:
setup_and_retrieval = RunnableParallel(
    {
        "context":retriever,
        "question":RunnablePassthrough()
    }
)

In [21]:
chain = setup_and_retrieval | prompt | llm_model | StrOutputParser()

In [22]:
chain.invoke("siapa nama presiden dan wakil presiden sekarang yah")

'Presiden Indonesia sekarang adalah Prabowo Subianto dan Wakil Presidennya adalah Gibran Rakabuming Raka.'

### memasukkan data external dengan document loader

In [24]:
import pandas as pd

In [28]:
DATA_PATH = '../notebooks/data/20250807110440 - laporan_nilaimhs.xlsx'

In [31]:
df = pd.read_excel(DATA_PATH)
df

,LAPORAN DAFTAR NILAI MAHASISWA,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,Program Studi S1 Bisnis Digital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Periode 2024 Genap,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mata kuliah,:,BAHASA INGGRIS 1,NaN,NaN,NaN,Nama Kelas,:,2BD01
4,Pengajar,:,"BETTY AMALIA\nKrishadi Nugroho, S.S, M.Si.",NaN,NaN,NaN,Sistem Kuliah,:,Reguler
...,...,...,...,...,...,...,...,...,...
687,34,110324035,FATHIMAH AZ'ZAHRA,88.08,4,A,NaN,NaN,NaN
688,35,110324036,AINI RATU MAWADDAH,82.29,3.7,A-,NaN,NaN,NaN
689,36,110324037,SALMA TSURAYYA AZ-ZAHRA,84.96,3.7,A-,NaN,NaN,NaN
690,37,110324038,MAHMUD,79.07,3.3,B+,NaN,NaN,NaN
